In [1]:
from datasets import load_dataset
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

/Users/yeungadrian/Library/Caches/pypoetry/virtualenvs/mlops-sjgU-X9u-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("dair-ai/emotion", "split")

In [3]:
labels = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}

In [4]:
embedding_model = TextEmbedding()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 20321.24it/s]


In [3]:
client = QdrantClient("http://localhost:6333")
client.create_collection(
    collection_name="dair-ai-emotion",
    vectors_config=models.VectorParams(
        size=384,  # Size of the embeddings generated by InstructorXL model
        distance=models.Distance.COSINE,
    ),
)

True

In [6]:
client.upload_points(  # upload_points is available as of qdrant-client v1.7.1
    collection_name="dair-ai-emotion",
    points=[
        models.PointStruct(
            id=id,
            vector=list(embedding_model.embed(doc["text"]))[0],
            payload={"text": doc["text"], "label": labels[doc["label"]]},
        )
        for id, doc in enumerate(dataset["validation"])
    ],
)

In [6]:
embedding_model = TextEmbedding("nomic-ai/nomic-embed-text-v1.5-Q")

Fetching 5 files: 100%|██████████| 5/5 [00:04<00:00,  1.22it/s]


In [9]:
client = QdrantClient("http://localhost:6333")
client.create_collection(
    collection_name="dair-ai-emotion-nomic",
    vectors_config=models.VectorParams(
        size=768,
        distance=models.Distance.COSINE,
    ),

)

True

In [10]:
client.upload_points(  # upload_points is available as of qdrant-client v1.7.1
    collection_name="dair-ai-emotion-nomic",
    points=[
        models.PointStruct(
            id=id,
            vector=list(embedding_model.embed(doc["text"]))[0],
            payload={"text": doc["text"], "label": labels[doc["label"]]},
        )
        for id, doc in enumerate(dataset["validation"])
    ],
)